<a href="https://colab.research.google.com/github/SrijaG29/questions_prac/blob/main/Stackoverflowpractise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
----------------------------------------------------Users-------------------------------------

Users----------------------------------

package com.srija.stackoverflow;

import com.fasterxml.jackson.annotation.JsonManagedReference;
import jakarta.persistence.*;
import jakarta.validation.constraints.Size;

import java.util.List;

@Entity
public class Users {
    @Id
    @GeneratedValue
    private Integer user_id;
    @Column(unique = true, nullable = false)
    private String username;
    @Column(unique = true, nullable = false)
    private String email;
    @Size(min = 8, message = "Password must be at least 8 characters long")
    @Column(nullable = false)
    private String password;

    @OneToMany(
            mappedBy = "users",
            cascade = CascadeType.ALL
    )
    @JsonManagedReference
    private List<Answers> answer;

    public Integer getUser_id() {
        return user_id;
    }

    public void setUser_id(Integer user_id) {
        this.user_id = user_id;
    }

    public String getUsername() {
        return username;
    }

    public void setUsername(String username) {
        this.username = username;
    }

    public String getEmail() {
        return email;
    }

    public void setEmail(String email) {
        this.email = email;
    }

    public @Size(min = 8, message = "Password must be at least 8 characters long") String getPassword() {
        return password;
    }

    public void setPassword(@Size(min = 8, message = "Password must be at least 8 characters long") String password) {
        this.password = password;
    }

    public List<Answers> getAnswer() {
        return answer;
    }

    public void setAnswer(List<Answers> answer) {
        this.answer = answer;
    }

    public Users() {
    }

}

Userscontorller--------------------------------------


package com.srija.stackoverflow;

import org.apache.catalina.User;
import org.springframework.web.bind.annotation.*;

import java.util.List;
import java.util.stream.Collectors;

import static java.util.stream.Nodes.collect;

@RestController
public class Userscontroller {

    private UsersService usersService;

    public Userscontroller(UsersService usersService) {
        this.usersService = usersService;
    }

    @PostMapping("/usercreate")
    public UsersresponseDto createuser(
            @RequestBody Users users
    ){
        return this.usersService.createuser(users);
    }


//    Finding all users
    @GetMapping("/usersdetails")
    public List<UsersresponseDto> findall(){
        return usersService.findall();
    }


//    Finding by username
    @GetMapping("/userdetails/username/{user-name}")
    public UsersresponseDto findbyusername(
            @PathVariable("username") String username
    ){
        return usersService.findbyusername(username);

    }

    @GetMapping("/userdetails/email/{email}")
    public UsersresponseDto finbyemail(
            @PathVariable("email") String email
    ){
        return usersService.findbyusername(email);
    }


}

UsersRepository--------------------------------------

package com.srija.stackoverflow;

import org.apache.catalina.User;
import org.springframework.data.jpa.repository.JpaRepository;

import java.util.Optional;

public interface UsersRepository extends JpaRepository<Users,Integer> {

    Users findByName(String username);
    Users findByEmail(String email);

}

UsersMapper-----------------------------------------

package com.srija.stackoverflow;

import org.springframework.stereotype.Service;

@Service
public class UsersMapper {
    public UsersresponseDto userresponse(Users user){
        return new UsersresponseDto(user.getUsername(),user.getEmail());
    }

    public UsersresponseDto tofindusers(Users user){
        return new UsersresponseDto(user.getUsername(),user.getEmail());
    }
}

UsersresponseDto--------------------------------------

package com.srija.stackoverflow;

public record UsersresponseDto(
        String username,
        String email
) {
}

UsersService------------------------------------------

package com.srija.stackoverflow;

import org.springframework.stereotype.Service;
import org.springframework.web.bind.annotation.PathVariable;
import org.springframework.web.bind.annotation.RequestBody;

import java.util.ArrayList;
import java.util.List;
import java.util.stream.Collectors;

@Service
public class UsersService {
    private final UsersRepository usersRepository;
    private final UsersMapper usersMapper;

    public UsersService(UsersRepository usersRepository, UsersMapper usersMapper) {
        this.usersRepository = usersRepository;
        this.usersMapper = usersMapper;
    }

    public UsersresponseDto createuser(
            Users users
    ){
        var ur = usersRepository.save(users);
        return usersMapper.userresponse(ur);
    }

    public List<UsersresponseDto> findall() {
        List<Users> usersList = usersRepository.findAll(); // Get all users from the repository
        List<UsersresponseDto> responseDtoList = new ArrayList<>();

        for (Users user : usersList) {
            UsersresponseDto dto = usersMapper.tofindusers(user); // Map each user to UsersresponseDto
            responseDtoList.add(dto); // Add the DTO to the response list
        }

        return responseDtoList; // Return the final list of DTOs
    }


    public UsersresponseDto findbyusername(
            String username
    ){
        Users user =  UsersRepository.findByName(username);
        return usersMapper.tofindusers(user);

    }

    public UsersresponseDto finbyemail(
            @PathVariable("email") String email
    ){
        Users user =  UsersRepository.findByEmail(email);
        return usersMapper.tofindusers(user);
    }


}

-------------------------------------------------------------Topics--------------------------------------------------

Topics---------------------------------------------------

package com.srija.stackoverflow;

import com.fasterxml.jackson.annotation.JsonBackReference;
import com.fasterxml.jackson.annotation.JsonManagedReference;
import com.srija.stackoverflow.Questions;
import jakarta.persistence.*;

import java.util.List;

@Entity
public class Topics {
    @Id
    @GeneratedValue
    private Integer topic_id;
    private String topic_name;

    @ManyToMany
    @JoinTable(
            name = "topics_questions",
            joinColumns = @JoinColumn(name = "topic_id"),
            inverseJoinColumns = @JoinColumn(name = "question_id")
    )
    @JsonManagedReference
    private List<Questions> questions;

    public Topics() {
    }

    public Integer getTopic_id() {
        return topic_id;
    }

    public void setTopic_id(Integer topic_id) {
        this.topic_id = topic_id;
    }

    public String getTopic_name() {
        return topic_name;
    }

    public void setTopic_name(String topic_name) {
        this.topic_name = topic_name;
    }

    public List<Questions> getQuestions() {
        return questions;
    }

    public void setQuestions(List<Questions> questions) {
        this.questions = questions;
    }
}

TopicsController----------------------------------------

package com.srija.stackoverflow;

import org.springframework.web.bind.annotation.*;

import java.util.List;

@RestController
public class TopicsController {
    private final TopicsRepository topicsRepository;

    public TopicsController(TopicsRepository topicsRepository) {
        this.topicsRepository = topicsRepository;
    }

    @PostMapping("/topics")
    public Topics creatingtopic(
            @RequestBody Topics topic
    ){
        return topicsRepository.save(topic);
    }

    @GetMapping("/alltopics")
    public List<Topics> findalltopics(){
        return topicsRepository.findAll();
    }

}

TopicsRepository-----------------------------------

package com.srija.stackoverflow;

import org.springframework.data.jpa.repository.JpaRepository;

public interface TopicsRepository extends JpaRepository<Topics, Integer> {
}

----------------------------------------------Questions--------------------------------------------------------

Questions-------------------------------

package com.srija.stackoverflow;

import com.fasterxml.jackson.annotation.JsonBackReference;
import com.fasterxml.jackson.annotation.JsonManagedReference;
import jakarta.persistence.*;
import org.springframework.boot.autoconfigure.kafka.KafkaProperties;

import java.util.List;

@Entity
public class Questions {
    @Id
    @GeneratedValue
    private Integer question_id;
    private String question;

    @OneToMany(
            mappedBy = "questions",
            cascade = CascadeType.ALL
    )
    @JsonManagedReference
    private List<Answers> answer;

    @ManyToMany(
            mappedBy = "questions",
            cascade = CascadeType.ALL
    )
    @JsonBackReference
    private List<Topics> topics;

    public Integer getQuestion_id() {
        return question_id;
    }

    public void setQuestion_id(Integer question_id) {
        this.question_id = question_id;
    }

    public String getQuestion() {
        return question;
    }

    public void setQuestion(String question) {
        this.question = question;
    }

    public List<Answers> getAnswer() {
        return answer;
    }

    public void setAnswer(List<Answers> answer) {
        this.answer = answer;
    }

    public List<Topics> getTopics() {
        return topics;
    }

    public void setTopics(List<Topics> topics) {
        this.topics = topics;
    }

    public Questions() {
    }



}

QuestionsRepository-------------------------

package com.srija.stackoverflow;

import org.springframework.data.jpa.repository.JpaRepository;

import java.util.Optional;

public interface QuestionsRepository extends JpaRepository<Questions,Integer> {

    Questions findByQuestion(String que);
}


---------------------------------------------Answers-------------------------------------------

Answers-----------------------------------------

package com.srija.stackoverflow;

import com.fasterxml.jackson.annotation.JsonBackReference;
import jakarta.persistence.*;

@Entity
public class Answers {
    @Id
    @GeneratedValue
    private Integer answer_id;
    private String answer;

    @ManyToOne
    @JoinColumn(
            name = "question_id"
    )
    @JsonBackReference
    private Questions questions;

    @ManyToOne
    @JoinColumn (
            name = "user_id"
    )
    @JsonBackReference
    private Users users;

    public Integer getAnswer_id() {
        return answer_id;
    }

    public void setAnswer_id(Integer answer_id) {
        this.answer_id = answer_id;
    }

    public String getAnswer() {
        return answer;
    }

    public void setAnswer(String answer) {
        this.answer = answer;
    }

    public Questions getQuestions() {
        return questions;
    }

    public void setQuestions(Questions questions) {
        this.questions = questions;
    }

    public Users getUsers() {
        return users;
    }

    public void setUsers(Users users) {
        this.users = users;
    }

    public Answers() {
    }



}

AnswersController---------------------------------

package com.srija.stackoverflow;

import org.springframework.web.bind.annotation.GetMapping;
import org.springframework.web.bind.annotation.PathVariable;
import org.springframework.web.bind.annotation.PostMapping;
import org.springframework.web.bind.annotation.RestController;

import java.util.List;

@RestController
public class AnswersController {

    private AnswersRepository answersRepository;

    public AnswersController(AnswersRepository answersRepository) {
        this.answersRepository = answersRepository;
    }

    @PostMapping("/answers")
    public Answers creatingasnwer(
            @PathVariable Answers answers
    ){
        return answersRepository.save(answers);
    }

    @GetMapping("/answers/{question}")
    public List<Answers> findallanswers(
            @PathVariable("question") String que
    ){
        Questions questionEntity = QuestionsRepository.findByQuestion(que);
        return answersRepository.findByQuestionId(questionEntity.getQuestion_id());
    }


}

AnswersRepository----------------------------------

package com.srija.stackoverflow;

import org.springframework.data.jpa.repository.JpaRepository;

import java.util.List;

public interface AnswersRepository extends JpaRepository<Answers,Integer> {
    List<Answers> findByQuestionId(Integer questionId);
}
